In [1]:
import sys


no_of_cpu = 8
max_cores = 8
executor_mem = '56g'



Job_Name = 'Dev Nandan :SPA Interaction'


from pyspark.mllib.stat import Statistics
from pyspark.sql.functions import asc,lit
#warnings.filterwarnings('error')
import pyspark
from datetime import datetime,timedelta
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as F

conf = pyspark.SparkConf()
#import numpy
import calendar
#import pandas as pd
#import simplejson as json
#import pandas as pd
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
from pyspark.sql.functions import lit
import simplejson as json
import json, pprint, requests
#es_nodes = '10.35.12.5'
#es_nodes = '10.35.12.6'
es_nodes = '10.35.12.194'#,10.35.12.6,10.35.12.5
es_nodes_temp='10.35.12.194'
es_port = '5432'
es_user = 'gpanalytics'
es_pwd = ''
mesos_ip = 'mesos://10.33.195.18:5050'#'mesos://10.35.12.5:5050'

spark.stop() #############NEED TO COMMENT THIS SPARK.STOP WHEN RUNNING THROUGH SHELL###############################

conf.setMaster(mesos_ip)

conf.set('spark.executor.cores',no_of_cpu) ### change 1
#conf.set('spark.memory.fraction','.2')
conf.set('spark.executor.memory',executor_mem) 
conf.set('spark.es.scroll.size','10000')
conf.set('spark.network.timeout','600s')
conf.set('spark.sql.crossJoin.enabled', 'true')

conf.set('spark.ui.port','4052')

conf.set('spark.executor.heartbeatInterval','60s')
conf.set("spark.driver.cores","4")
conf.set("spark.driver.extraJavaOptions","-Xmx4g -Xms4g")

#conf.set("spark.shuffle.blockTransferService", "nio"); 
conf.set("spark.files.overwrite","true");
conf.set("spark.kryoserializer.buffer", "70"); 
conf.set("spark.driver.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.executor.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer"); 
conf.set("spark.broadcast.compress", "true"); 
conf.set("spark.shuffle.compress", "true"); 
conf.set("spark.shuffle.spill.compress", "true");
conf.set("spark.app.name", Job_Name)
#conf.set("spark.io.compression.codec","org.apache.spark.io.LZ4CompressionCodec");
#conf.set("spark.sql.inMemoryColumnarStorage.compressed", "true"); 
from pyspark.sql.functions import broadcast

conf.set('spark.driver.memory','20g') ### change 2
conf.set('spark.cores.max',max_cores) ### change 3
conf.set('spark.sql.shuffle.partitions','400')
#conf.set('spark.sql.crossJoin.enabled', 'true')
conf.set('es.nodes',es_nodes)
conf.set('es.port',es_port)
conf.set('es.nodes.wan.only','true')
conf.set("spark.sql.autoBroadcastJoinThreshold",-1)

#conf.set('spark.es.net.http.auth.user','Spark')
#conf.set('spark.es.net.http.auth.pass','Jarkpet1Sap3')
conf.set('spark.num.executors','4')
conf.set('spark.debug.maxToStringFields', 200)

conf.set('spark.es.net.http.auth.user', es_user)
conf.set('spark.es.net.http.auth.pass', es_pwd)

conf.set('spark.es.mapping.date.rich','false')
spark = pyspark.SparkContext(conf=conf)
sqlContext = SQLContext(spark)

# Load Data into PySpark DataFrames
# Prodcom Data Frame
import json, pprint, requests
import pyspark.sql.functions as sf
from pyspark.sql.types import *


import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType



Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
15982,None,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql.functions import hash
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
import psycopg2
import time
from time import gmtime, strftime
import pytz
starttime = time.time()
start_time = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d %H:%M:%S.%f")
start_time

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'2023-12-13 22:48:46.795194'

In [3]:
prod_url = "jdbc:postgresql://10.35.12.194:5432/gpadmin"
prod_host = '10.35.12.194'
prod_port = '5432'
prod_dbname = 'gpadmin'
user_prod="gpspark"
pwd_prod="spark@456"
dbschema="public"


prod_gpdb_spark_options ={
    "url": "jdbc:postgresql://{host}:{port}/{dbname}".format(host=prod_host,port=prod_port, dbname=prod_dbname),
    "user": "{user}".format(user=user_prod),
    "password": "{password}".format(password=pwd_prod)
} 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
def load_gpdb_jdbc(col_str,dbtable,dbschema,col_name=None,time_filter=None,partitionColumn="row_num"):
    """
    This is used to read gpdb with filter for columns and can apply other filter(date,values).
    Time filter contains startdate,enddate
    """
    gscPythonOptions = {
                        "url": prod_url,
                        "user": user_prod,
                        "password": pwd_prod,
                        "dbschema": dbschema,
                        "dbtable": dbtable,
                        "partitionColumn":partitionColumn,
                        "partitions": 8,
                        "server.port":"1150-1170"}
   
    if time_filter:
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .selectExpr(col_str).drop_duplicates().filter(col(col_name).between(to_timestamp(lit(time_filter['start_date']),
                                                                 format='yyyy-MM-dd'),
                                                    to_timestamp(lit(time_filter['end_date']),
                                                                 format='yyyy-MM-dd')))
    else :
        data = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                .selectExpr(col_str).drop_duplicates()
   
    return data

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
def get_latest_progress(table_name,source):
    
    try: 
        gscPythonOptions = {
                 "url":prod_url ,
                 "user":user_prod ,
                 "password": pwd_prod,
                 "dbschema":"customermart",
                 "dbtable": "progress",#table change
                 "server.port":"1150-1170"} 
        
        # this query will fetch till what date we have inserted the records in target
        last_run= sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
            .select('table_name','source','to_datetime').filter(col('table_name')==table_name)\
            .filter(col('source')==source).filter(col('status')=='Success')\
        .filter(col('records')!=0)
        
        Max_last_run =last_run.select(max('to_datetime')).first()[0]


        # if no record is available in progress then it will raise value error, in this case default value will be picked from except clause
        if Max_last_run is None:
            raise ValueError("No records found!")

        #print("Record Found. Progress updated till {}".format(Max_last_run))

    except Exception as E: 
        Max_last_run = datetime.datetime(2001,1, 1, 0, 0)  # this is the default start date when no record is present in progress for customer_demographics table
        print("Executed_except",repr(E))
    return Max_last_run

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Generic function to save the progress from last run into Progress table
def update_progress(table_name,source,time_filter,records,start_time,starttime,status):
    output_index = "progress"
    schema = "customermart"


    import sys
    import time
    try:
        


        df_progress= sqlContext.createDataFrame([(table_name,source,
                                       time_filter['start_date'].strftime("%Y-%m-%d %H:%M:%S"),
                                         time_filter['end_date'].strftime("%Y-%m-%d %H:%M:%S"),records)]
                                     ,['table_name', 'source','from_datetime','to_datetime','records'])\
        .withColumn('start_time',lit(start_time)).withColumn('end_time',current_timestamp())\
                    .withColumn('executed_in_mins',lit((time.time() - starttime)/60))\
                                .withColumn('status',lit(status)).withColumn('remarks',lit(None))
    

        
        df_progress.write.format("greenplum")\
            .option("dbtable",output_index).option('dbschema',schema)\
            .option("server.port","1150-1170").option("url",prod_url)\
            .option("user", user_prod).option("password",pwd_prod).mode('append').save()
        
       
    except Exception as e:
        x = e
        print(x)
    else:
        x = 'success'
        print("Updated Progress for {table_name} from {source} until {to_datetime}. {records} records processed in this run".format(table_name=table_name, source=source,to_datetime=time_filter['end_date'].strftime("%Y-%m-%d %H:%M:%S"),records=records))
        print(x)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# Define the Generic update function which takes the records currently in temp table (created with prefix 1 to original customercoe table)
# This temp table has similar schema as original table but no json field and would not contain duplicates
# The function would first try to insert the records in original table. It that fails then it will upsert the records
# Since there is no direct upsert query an update and insert query with where clause is used
# This funciton just runs the SQL queries in GPDB and does not use spark

def update(conn_to,table_name,gpdb_spark_options = prod_gpdb_spark_options,schema="customermart"):
    conn_to.rollback()
    
    source='interaction_journal'
    table_dict = {
    "interaction_journal": #table_name
              {
                  "out_columns" : ['source_app','interaction_type','source_system_customer_id', 'policy_no','certificate_no','interaction_date','reference_num','transaction_id','ticket_no','mobile','email','awb_no',
                                             'customer_name','deposition','sub_deposition','callback_date','remarks','language_pref','language_barrier',
                                             'lost_reason','competitor_name','ticket_close_date','ticket_tat','ticket_status','ticket_source','open_status',
                                             'link_status','print_flag','courier','delivery_return_date','recieved_by'],
                  "primary_key" : ['source_app','interaction_type','source_system_customer_id', 'policy_no','certificate_no'],
                  "interaction_journal" : #source/subsegment
                      { 
                          "update_fields" : ['interaction_date','reference_num','transaction_id','ticket_no','mobile','email','awb_no',
                                             'customer_name','deposition','sub_deposition','callback_date','remarks','language_pref','language_barrier',
                                             'lost_reason','competitor_name','ticket_close_date','ticket_tat','ticket_status','ticket_source','open_status',
                                             'link_status','print_flag','courier','delivery_return_date','recieved_by']
                      }
              }
    }
    
    
    update_fields = table_dict.get(table_name).get(source).get("update_fields")
    primary_key = table_dict.get(table_name).get("primary_key")
    
    # loading data from temp table to get column names
    #out_columns=['source_system_name','source_system_customer_id','activity_id','activity_date','communication_type','communication_sub_type','value','verification_flag','verification_source','date_reported','last_updated','preference_flag','dnd_flag' ]     
    df_1 = sqlContext.read.format("jdbc").options(**gpdb_spark_options).option("dbtable","customermart.{}_staging".format(table_name)).load()
    out_columns = df_1.columns
    out_columns = [col for col in out_columns if "json" not in col ]
#     out_columns
    #out_columns = table_dict.get(table_name).get("out_columns")

    cur_to = conn_to.cursor()  

    # this query insert records from temp table to target table when all the records in temp table are new and not present in target
    only_insert_query = """insert into {schema}.{table_name} ({out_columns}) select {out_columns} from {schema}.{table_name}_staging """.format(schema=schema,table_name=table_name,out_columns = ",".join(out_columns))
    
    # this query will be executed to update the records that are already present in target, new values will be assigned from temp table
    update_query = """UPDATE {schema}.{table_name} orig
                          SET
                            {update_fields}
                          FROM
                            {schema}.{table_name}_staging temp
                           WHERE 
                            {primary_key}
                       """.format(schema=schema,table_name=table_name,primary_key= " and ".join([ "orig.{key} = temp.{key}".format(key=key) for key in primary_key]),update_fields = " , ".join([ "{key} = temp.{key}".format(key=key) for key in update_fields]))
    
    # this query will be executed after update query to insert the remaining records
    insert_query = """ INSERT INTO {schema}.{table_name} ({out_columns})
                           SELECT {out_columns}
                           FROM
                             {schema}.{table_name}_staging temp
                           WHERE
                             NOT EXISTS (
                             SELECT 1 FROM {schema}.{table_name} orig WHERE 
                            {primary_key}
                            )
                        """.format(schema=schema,table_name=table_name,out_columns = ",".join(out_columns),primary_key = " and ".join([ "orig.{key} = temp.{key}".format(key=key) for key in primary_key]))

    try:
        print("Inside try segment")
        cur_to.execute(only_insert_query)
        print("Executed Only insert query")
#         update_progress(table_name,source,time_filter,records)
    except Exception as e:
        print("Inside except segment")
        conn_to.rollback()
        cur_to.execute(update_query)
        print("Executed update query")
        cur_to.execute(insert_query)
        print("Executed insert query")
#         update_progress(table_name,source,time_filter,records)
    conn_to.commit()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# #Adding DMS details

In [ ]:
starttime = time.time()
start_time = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d %H:%M:%S.%f")
start_time

In [ ]:
table_name='interaction_journal'
source='DMS'

In [ ]:
latest_progress=get_latest_progress(table_name,source)
# dates in string format
str_d1 = latest_progress.strftime("%Y-%m-%d")
str_d2 = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d")

# convert string to date object
d1 = datetime.datetime.strptime(str_d1, "%Y-%m-%d")
d2 = datetime.datetime.strptime(str_d2, "%Y-%m-%d")

# difference between dates in timedelta
delta = d2 - d1
day=delta.days-1#currently T-2 change 2 to 1 for T-1

time_filter={'start_date':latest_progress,'end_date':latest_progress+datetime.timedelta(days=day)}
time_filter

In [ ]:
gscPythonOptions = {
         "url": prod_url,
         "user": user_prod,
         "password": pwd_prod,
         "dbschema":"public",
         "dbtable": "dms_cmpt_tra_lcu_production",
         "server.port": "1151-1160"} 

# Get the data  from  Table

dms_df = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
    .selectExpr('vuniqueno','vpolicyno as policy_no','txt_certificate_no as certificate_no','proposal_no as reference_num','txt_customer_id as source_system_customer_id',
            'transactionid as transaction_id','print_flag','dispatchdate','courier','airwaybillno as awb_no','courierstatus',
            'courierstatusdate','receivedby as recieved_by','return_reason as remarks','return_reason_code','c_customeremail as email',
            'filegenerationdate','bitly_mobile_no as mobile','bitly_sms_date','istatus','dtissuancecreated','vstatusdate').dropDuplicates()\
   .filter(col("load_date")>=(to_date(lit(time_filter['start_date']), format='yyyy-MM-dd')))\
   .filter(col("load_date")<=(to_date(lit(time_filter['end_date']), format='yyyy-MM-dd')))\
   .filter(col("policy_no").isNotNull())\
   .filter((col('dispatchdate').isNotNull()) | (col('filegenerationdate').isNotNull()) | (col('bitly_sms_date').isNotNull()))

In [ ]:
dms_df=dms_df.dropDuplicates()

In [ ]:
dms_hard=dms_df.filter(~col('istatus').isin([8,10])).select('policy_no','certificate_no','source_system_customer_id','transaction_id','awb_no','print_flag','dispatchdate','courier','recieved_by','remarks', 'reference_num')

In [ ]:
dms_hard=dms_hard.withColumn('source_app',lit('dms')).withColumn('interaction_type',lit('courier'))

In [ ]:
dms_hard=dms_hard.withColumnRenamed('dispatchdate','interaction_date')

In [ ]:
dms_hard=dms_hard.withColumn('ticket_no',lit(None).cast('string'))\
.withColumn('mobile',lit(None).cast('string'))\
.withColumn('email',lit(None).cast('string'))\
.withColumn('customer_name',lit(None).cast('string'))\
.withColumn('deposition',lit(None).cast('string'))\
.withColumn('sub_deposition',lit(None).cast('string'))\
.withColumn('callback_date',lit(None).cast('date'))\
.withColumn('language_pref',lit(None).cast('string'))\
.withColumn('language_barrier',lit(None).cast('string'))\
.withColumn('lost_reason',lit(None).cast('string'))\
.withColumn('competitor_name',lit(None).cast('string'))\
.withColumn('ticket_close_date',lit(None).cast('date'))\
.withColumn('ticket_tat',lit(None).cast('int'))\
.withColumn('ticket_status',lit(None).cast('string'))\
.withColumn('ticket_source',lit(None).cast('string'))\
.withColumn('open_status',lit(None).cast('string'))\
.withColumn('link_status',lit(None).cast('string'))\
.withColumn('delivery_return_date',lit(None).cast('date'))

In [ ]:
dms_hard=dms_hard.select('source_app','interaction_type','interaction_date','policy_no','certificate_no','source_system_customer_id','reference_num','transaction_id','ticket_no',
'mobile','email','awb_no','customer_name','deposition','sub_deposition','callback_date','remarks','language_pref','language_barrier','lost_reason',
'competitor_name','ticket_close_date','ticket_tat','ticket_status','ticket_source','open_status','link_status','print_flag','courier','delivery_return_date',
'recieved_by')

In [ ]:
dms_email=dms_df.filter(col('istatus')=='8').select('policy_no','certificate_no','source_system_customer_id','transaction_id','email','filegenerationdate', 'reference_num')

In [ ]:
dms_email=dms_email.withColumn('source_app',lit('dms')).withColumn('interaction_type',lit('email'))

In [ ]:
dms_email=dms_email.withColumnRenamed('filegenerationdate','interaction_date')

In [ ]:
dms_email=dms_email.withColumn('ticket_no',lit(None).cast('string'))\
.withColumn('mobile',lit(None).cast('string'))\
.withColumn('awb_no',lit(None).cast('string'))\
.withColumn('customer_name',lit(None).cast('string'))\
.withColumn('deposition',lit(None).cast('string'))\
.withColumn('sub_deposition',lit(None).cast('string'))\
.withColumn('callback_date',lit(None).cast('date'))\
.withColumn('remarks',lit(None).cast('date'))\
.withColumn('language_pref',lit(None).cast('string'))\
.withColumn('language_barrier',lit(None).cast('string'))\
.withColumn('lost_reason',lit(None).cast('string'))\
.withColumn('competitor_name',lit(None).cast('string'))\
.withColumn('ticket_close_date',lit(None).cast('date'))\
.withColumn('ticket_tat',lit(None).cast('int'))\
.withColumn('ticket_status',lit(None).cast('string'))\
.withColumn('ticket_source',lit(None).cast('string'))\
.withColumn('open_status',lit(None).cast('string'))\
.withColumn('link_status',lit(None).cast('string'))\
.withColumn('print_flag',lit(None).cast('string'))\
.withColumn('courier',lit(None).cast('string'))\
.withColumn('delivery_return_date',lit(None).cast('date'))\
.withColumn('recieved_by',lit(None).cast('string'))

In [ ]:
dms_email=dms_email.select('source_app','interaction_type','interaction_date','policy_no','certificate_no','source_system_customer_id','reference_num','transaction_id','ticket_no',
'mobile','email','awb_no','customer_name','deposition','sub_deposition','callback_date','remarks','language_pref','language_barrier','lost_reason',
'competitor_name','ticket_close_date','ticket_tat','ticket_status','ticket_source','open_status','link_status','print_flag','courier','delivery_return_date',
'recieved_by')

In [ ]:
dms_sms=dms_df.filter(col('istatus')=='10').select('policy_no','certificate_no','source_system_customer_id','transaction_id','mobile','bitly_sms_date', 'reference_num')

In [ ]:
dms_sms=dms_sms.withColumn('source_app',lit('dms')).withColumn('interaction_type',lit('sms'))

In [ ]:
dms_sms=dms_sms.withColumnRenamed('bitly_sms_date','interaction_date')

In [ ]:
dms_sms=dms_sms.withColumn('ticket_no',lit(None).cast('string'))\
.withColumn('email',lit(None).cast('string'))\
.withColumn('awb_no',lit(None).cast('string'))\
.withColumn('customer_name',lit(None).cast('string'))\
.withColumn('deposition',lit(None).cast('string'))\
.withColumn('sub_deposition',lit(None).cast('string'))\
.withColumn('callback_date',lit(None).cast('date'))\
.withColumn('remarks',lit(None).cast('string'))\
.withColumn('language_pref',lit(None).cast('string'))\
.withColumn('language_barrier',lit(None).cast('string'))\
.withColumn('lost_reason',lit(None).cast('string'))\
.withColumn('competitor_name',lit(None).cast('string'))\
.withColumn('ticket_close_date',lit(None).cast('date'))\
.withColumn('ticket_tat',lit(None).cast('int'))\
.withColumn('ticket_status',lit(None).cast('string'))\
.withColumn('ticket_source',lit(None).cast('string'))\
.withColumn('open_status',lit(None).cast('string'))\
.withColumn('link_status',lit(None).cast('string'))\
.withColumn('print_flag',lit(None).cast('string'))\
.withColumn('courier',lit(None).cast('string'))\
.withColumn('delivery_return_date',lit(None).cast('date'))\
.withColumn('recieved_by',lit(None).cast('string'))

In [ ]:
dms_sms=dms_sms.select('source_app','interaction_type','interaction_date','policy_no','certificate_no','source_system_customer_id','reference_num','transaction_id','ticket_no',
'mobile','email','awb_no','customer_name','deposition','sub_deposition','callback_date','remarks','language_pref','language_barrier','lost_reason',
'competitor_name','ticket_close_date','ticket_tat','ticket_status','ticket_source','open_status','link_status','print_flag','courier','delivery_return_date',
'recieved_by')

In [ ]:
df_union_1=dms_hard.union(dms_email)
df_union_2=df_union_1.union(dms_sms)

In [ ]:
from pyspark.sql.functions import unix_timestamp, from_unixtime
df_union_2=df_union_2.withColumn('interaction_date', from_unixtime(unix_timestamp('interaction_date', 'dd/MM/yyyy')).cast('timestamp'))

In [ ]:
df_union_2=df_union_2.select([col(c).cast("string") for c in df_union_2.columns])
df_union_2=df_union_2.withColumn('interaction_date', col('interaction_date').cast(DateType()))\
.withColumn('delivery_return_date', col('delivery_return_date').cast(DateType()))\
.withColumn('reference_num', col('reference_num').cast('long'))\
.withColumn('callback_date', col('callback_date').cast(DateType()))\
.withColumn('ticket_close_date', col('ticket_close_date').cast(DateType()))\
.withColumn('ticket_tat', col('ticket_tat').cast('long'))
df_union_2=df_union_2.filter(col('interaction_date').isNotNull()).filter(col('source_system_customer_id').isNotNull())
df_union_2=df_union_2.withColumn("certificate_no", coalesce(col("certificate_no"), lit('00000000')))

In [ ]:
primary_key = ['source_app','interaction_type','source_system_customer_id', 'policy_no','certificate_no']

from pyspark.sql.window import Window

df_union_2 = df_union_2.withColumn("rn", F.row_number()
        .over(Window.partitionBy(primary_key)
        .orderBy(F.col("interaction_date").desc())))

df_union_2 = df_union_2.filter(F.col("rn") == 1).drop("rn")

In [ ]:
df_union_2.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
records=df_union_2.count()
records #23347632

In [ ]:
#df_union_2=df_union_2.limit(20000)

In [ ]:
output_index = "interaction_journal_staging"
schema = "customermart"


import sys
try:

    df_union_2.write.format("greenplum")\
    .option("dbtable",output_index).option('dbschema',schema)\
    .option("server.port","1150-1170").option("url",prod_url)\
    .option("user", user_prod).option("password",pwd_prod).mode('overwrite').save()


except Exception as e:
    x = e
    print(x)
else:
    x = 200 #success
    print(x)

In [ ]:
table_name="interaction_journal"
conn_prod = psycopg2.connect(host=prod_host, port=prod_port, user=user_prod, password=pwd_prod, dbname=prod_dbname)
conn_to=conn_prod
status=''

import sys
try:
    update(conn_to,table_name,prod_gpdb_spark_options, schema="customermart")
except Exception as e:
    x = e
    print(x)
    status='Failed'
else:
    x = 'success'
    print(x)
    status='Success'

In [ ]:
table_name='interaction_journal'
source='DMS'


if  x == 'success':
    update_progress(table_name,source,time_filter,records,start_time,starttime,'Success')
    print('SUCCESS')
else:
    update_progress(table_name,source,time_filter,'0',start_time,starttime,'Failed')
    print('FAILED')
    
df_union_2.unpersist()

# ICRM

In [ ]:
starttime = time.time()
start_time = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d %H:%M:%S.%f")
start_time

In [ ]:
table_name='interaction_journal'
source='ICRM'

In [ ]:
latest_progress=get_latest_progress(table_name,source)
# dates in string format
str_d1 = latest_progress.strftime("%Y-%m-%d")
str_d2 = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d")

# convert string to date object
d1 = datetime.datetime.strptime(str_d1, "%Y-%m-%d")
d2 = datetime.datetime.strptime(str_d2, "%Y-%m-%d")

# difference between dates in timedelta
delta = d2 - d1
day=delta.days-1#currently T-2 change 2 to 1 for T-1

time_filter={'start_date':latest_progress,'end_date':latest_progress+datetime.timedelta(days=day)}
time_filter

In [ ]:
gscPythonOptions = {"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin","user": "gpspark","password": "spark@456",
         "dbschema": "public","dbtable": "marketing_icrm_mph_policy",
         "partitionColumn":"row_num","partitions":5,"server.port":"1107"} 

icrm_policy = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
.select('ID','NAME', 'customer_code')\
.filter(col("load_date")>=(to_date(lit(time_filter['start_date']), format='yyyy-MM-dd')))\
.filter(col("load_date")<=(to_date(lit(time_filter['end_date']), format='yyyy-MM-dd')))


icrm_policy_1 = icrm_policy.withColumnRenamed('name', 'POLICY_NO')\
                           .withColumnRenamed('id', 'POLICY_ID')\
                           .withColumnRenamed('customer_code', 'source_system_customer_id')

icrm_policy_1A = icrm_policy_1.groupBy('POLICY_ID').agg(sf.max('POLICY_NO').alias('POLICY_NUMBER'), sf.max('source_system_customer_id').alias('source_system_customer_id'))
icrm_policy_2 = icrm_policy_1A.repartition("POLICY_NUMBER")
#sagar to join with policy index (search policy wise) policy no and create two indexex policy ticket and policy mobile ticket
#rccn_icrm_policy_1 = distinct_policy.join(icrm_policy_2, 'POLICY_NUMBER','inner')
rccn_icrm_policy_1=icrm_policy_2

In [ ]:
gscPythonOptions = {"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin","user": "gpspark","password": "spark@456",
         "dbschema": "public","dbtable": "marketing_icrm_cases",
         "partitionColumn":"row_num","partitions":1,"server.port":"1107"} 

icrm_cases = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
.select('ID','NAME','DATE_ENTERED','CASE_CLOSURE_DATE','SERVICE_STATUS','SOURCE','WORK_NOTE','QUEUE_NAME','INTERACTION_CATEGORY','DESCRIPTION','COMPLAINT_STATUS')\
.filter(col("load_date")>=(to_date(lit(time_filter['start_date']), format='yyyy-MM-dd')))\
.filter(col("load_date")<=(to_date(lit(time_filter['end_date']), format='yyyy-MM-dd')))


In [ ]:
icrm_cases_1 = icrm_cases.withColumnRenamed('id', 'CASE_ID')\
                         .withColumnRenamed('name', 'INBOUND_TICKET')\
                         .withColumnRenamed('date_entered', 'TICKET_RAISED_DATE')\
                         .withColumnRenamed('case_closure_date', 'TICKET_CLOSED_DATE')\
                         .withColumnRenamed('service_status', 'TICKET_STATUS')\
                         .withColumnRenamed('source', 'SOURCE')\
                         .withColumnRenamed('queue_name', 'QUEUE_ID')\
                         .withColumnRenamed('interaction_category', 'SR_ID')\
                         .withColumn('TICKET_REMARK', substring(col("work_note"), 1, 255))\

In [ ]:
icrm_cases_2 = icrm_cases_1.groupBy('CASE_ID')\
                           .agg(sf.max('INBOUND_TICKET').alias('INBOUND_TICKET'),\
                                sf.max('TICKET_RAISED_DATE').alias('TICKET_RAISED_DATE'),\
                                sf.max('TICKET_CLOSED_DATE').alias('TICKET_CLOSED_DATE'),\
                                sf.max('TICKET_STATUS').alias('TICKET_STATUS'),\
                                sf.max('SOURCE').alias('SOURCE'),\
                                sf.max('TICKET_REMARK').alias('TICKET_REMARK'),\
                                sf.max('QUEUE_ID').alias('QUEUE_ID'),\
                                sf.max('DESCRIPTION').alias('TICKET_REASON'),\
                                sf.max('COMPLAINT_STATUS').alias('TICKET_COMPLAINT_STATUS'),\
                                sf.max('SR_ID').alias('SR_ID'))

In [ ]:
gscPythonOptions = {"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin","user": "gpspark","password": "spark@456",
         "dbschema": "public","dbtable": "marketing_icrm_mph_queues",
         "partitionColumn":"row_num","partitions":1,"server.port":"1107"} 

icrm_queues = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
.select('ID','NAME')\
.filter(col("load_date")>=(to_date(lit(time_filter['start_date']), format='yyyy-MM-dd')))\
.filter(col("load_date")<=(to_date(lit(time_filter['end_date']), format='yyyy-MM-dd')))

In [ ]:
icrm_queues_1 = icrm_queues.withColumnRenamed('name', 'TICKET_TYPE')\
                           .withColumnRenamed('id', 'QUEUE_ID') 

In [ ]:
icrm_queues_2 = icrm_queues_1.groupBy('QUEUE_ID').agg(sf.max('TICKET_TYPE').alias('TICKET_TYPE'))

In [ ]:
gscPythonOptions = {"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin","user": "gpspark","password": "spark@456",
         "dbschema": "public","dbtable": "marketing_icrm_mph_sr",
         "partitionColumn":"row_num","partitions":1,"server.port":"1107"} 

icrm_sr = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
.select('ID','NAME')

In [ ]:
icrm_sr_1 = icrm_sr.withColumnRenamed('name', 'SUB_TYPE')\
                   .withColumnRenamed('id', 'SR_ID')

In [ ]:
icrm_sr_2 = icrm_sr_1.groupBy('SR_ID').agg(sf.max('SUB_TYPE').alias('SUB_TYPE'))

In [ ]:
gscPythonOptions = {"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin","user": "gpspark","password": "spark@456",
         "dbschema": "public","dbtable": "marketing_icrm_mph_policy_cases_1_c",
         "partitionColumn":"row_num","partitions":1,"server.port":"1107"} 

icrm_policy_cases = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
.select('MPH_POLICY_CASES_1CASES_IDB', 'MPH_POLICY_CASES_1MPH_POLICY_IDA')

In [ ]:
icrm_policy_cases_1 = icrm_policy_cases.withColumnRenamed('mph_policy_cases_1cases_idb', 'CASE_ID')\
                                       .withColumnRenamed('mph_policy_cases_1mph_policy_ida', 'POLICY_ID')

In [ ]:
icrm_df = rccn_icrm_policy_1.join(icrm_policy_cases_1, 'POLICY_ID', 'left')

In [ ]:
rccn_icrm_policy_1.columns

In [ ]:
icrm_df_1 = icrm_df.join(icrm_cases_1, 'CASE_ID', 'left')

In [ ]:
icrm_df_2 = icrm_df_1.join(icrm_sr_2, 'SR_ID', 'left')

In [ ]:
icrm_df_3 = icrm_df_2.join(icrm_queues_2, 'QUEUE_ID', 'left')

In [ ]:
icrm_df_3A = icrm_df_3.withColumn('TICKET_RAISED_DATE', from_unixtime(unix_timestamp(col('TICKET_RAISED_DATE')), "yyyy-MM-dd"))\
.withColumn('TICKET_CLOSED_DATE', from_unixtime(unix_timestamp(col('TICKET_CLOSED_DATE')), "yyyy-MM-dd"))\
.drop('SR_ID')\
.drop('CASE_ID')\
.drop('POLICY_ID')\
.drop('QUEUE_ID')


In [ ]:
icrm_df_4=icrm_df_3A.withColumn('TAT',datediff(col("TICKET_CLOSED_DATE"),col("TICKET_RAISED_DATE")))

In [ ]:
icrm_df_4_1=icrm_df_4.withColumn('INBOUND_TICKET',when(col('INBOUND_TICKET').isNull(),lit('-')).otherwise(col('INBOUND_TICKET')))\
.withColumn('TICKET_RAISED_DATE',when(col('TICKET_RAISED_DATE').isNull(),lit('-')).otherwise(col('TICKET_RAISED_DATE')))\
.withColumn('TICKET_CLOSED_DATE',when(col('TICKET_CLOSED_DATE').isNull(),lit('-')).otherwise(col('TICKET_CLOSED_DATE')))\
.withColumn('TICKET_STATUS',when(col('TICKET_STATUS').isNull(),lit('-')).otherwise(col('TICKET_STATUS')))\
.withColumn('SOURCE',when(col('SOURCE').isNull(),lit('-')).otherwise(col('SOURCE')))\
.withColumn('TICKET_REMARK',when(col('TICKET_REMARK').isNull(),lit('-')).otherwise(col('TICKET_REMARK')))\
.withColumn('SUB_TYPE',when(col('SUB_TYPE').isNull(),lit('-')).otherwise(col('SUB_TYPE')))\
.withColumn('TICKET_TYPE',when(col('TICKET_TYPE').isNull(),lit('-')).otherwise(col('TICKET_TYPE')))\
.withColumn('TAT',when(col('TAT').isNull(),lit(0)).otherwise(col('TAT')))

In [ ]:
import time
ts = time.gmtime()
import pytz
# derive important parameters
IST = pytz.timezone('Asia/Kolkata')
current_date = datetime.datetime.now(IST).strftime("%Y-%m-%d")

In [ ]:
icrm_df_5=icrm_df_4_1.withColumnRenamed('INBOUND_TICKET','TICKET_NUMBER')\
                        .withColumnRenamed('TICKET_RAISED_DATE','TICKET_CREATED_DATE')\
                        .withColumnRenamed('TAT','TICKET_TAT_IN_DAYS')\
                        .withColumnRenamed('SUB_TYPE','TICKET_SUB_TYPE')\
                        .withColumnRenamed('SOURCE','TICKET_SOURCE')\
                        .withColumnRenamed('TAT','TICKET_TAT_IN_DAYS')\
                        .withColumnRenamed('TICKET_REMARK','TICKET_WORK_NOTE')\

In [ ]:
df_main=icrm_df_5.drop_duplicates()
df_main=df_main.withColumn('LOAD_DATE',lit(current_date))

In [ ]:
icrm_df=df_main.withColumn('TICKET_NUMBER',when(col('TICKET_NUMBER').isNull(),lit('-')).otherwise(col('TICKET_NUMBER')))\
.withColumn('TICKET_CREATED_DATE',when(col('TICKET_CREATED_DATE').isNull(),lit('-')).otherwise(col('TICKET_CREATED_DATE')))\
.withColumn('ticket_close_date',when(col('TICKET_CLOSED_DATE').isNull(),lit('-')).otherwise(col('TICKET_CLOSED_DATE')))\
.withColumn('ticket_status',when(col('TICKET_STATUS').isNull(),lit('-')).otherwise(col('TICKET_STATUS')))\
.withColumn('ticket_source',when(col('TICKET_SOURCE').isNull(),lit('-')).otherwise(col('TICKET_SOURCE')))\
.withColumn('TICKET_WORK_NOTE',when(col('TICKET_WORK_NOTE').isNull(),lit('-')).otherwise(col('TICKET_WORK_NOTE')))\
.withColumn('TICKET_SUB_TYPE',when(col('TICKET_SUB_TYPE').isNull(),lit('-')).otherwise(col('TICKET_SUB_TYPE')))\
.withColumn('TICKET_TYPE',when(col('TICKET_TYPE').isNull(),lit('-')).otherwise(col('TICKET_TYPE')))\
.withColumn('TICKET_TAT_IN_DAYS',when(col('TICKET_TAT_IN_DAYS').isNull(),lit(0)).otherwise(col('TICKET_TAT_IN_DAYS')))

In [ ]:
icrm_df=icrm_df.withColumnRenamed('TICKET_WORK_NOTE','remarks')\
.withColumnRenamed('TICKET_TYPE','deposition')\
.withColumnRenamed('TICKET_SUB_TYPE','sub_deposition')\
.withColumnRenamed('TICKET_TAT_IN_DAYS','ticket_tat')\

In [ ]:
icrm_df=icrm_df.withColumn('source_app',lit('ICRM')).withColumn('interaction_type',lit('Inbound Calling'))

In [ ]:
icrm_df=icrm_df.withColumnRenamed('TICKET_CREATED_DATE','interaction_date')\
.withColumnRenamed('POLICY_NUMBER','policy_no')\
.withColumnRenamed('TICKET_NUMBER','ticket_no')

In [ ]:
icrm_df=icrm_df.withColumn('certificate_no',lit(None).cast('string'))\
.withColumn('reference_num',lit(None).cast('long'))\
.withColumn('transaction_id',lit(None).cast('string'))\
.withColumn('mobile',lit(None).cast('string'))\
.withColumn('email',lit(None).cast('string'))\
.withColumn('awb_no',lit(None).cast('string'))\
.withColumn('customer_name',lit(None).cast('string'))\
.withColumn('callback_date',lit(None).cast('date'))\
.withColumn('language_pref',lit(None).cast('string'))\
.withColumn('language_barrier',lit(None).cast('string'))\
.withColumn('lost_reason',lit(None).cast('string'))\
.withColumn('competitor_name',lit(None).cast('string'))\
.withColumn('open_status',lit(None).cast('string'))\
.withColumn('link_status',lit(None).cast('string'))\
.withColumn('print_flag',lit(None).cast('string'))\
.withColumn('courier',lit(None).cast('string'))\
.withColumn('delivery_return_date',lit(None).cast('date'))\
.withColumn('recieved_by',lit(None).cast('string'))

In [ ]:
icrm_df=icrm_df.select([col(c).cast("string") for c in icrm_df.columns])
icrm_df=icrm_df.withColumn('interaction_date', col('interaction_date').cast(DateType()))\
.withColumn('delivery_return_date', col('delivery_return_date').cast(DateType()))\
.withColumn('reference_num', col('reference_num').cast('long'))\
.withColumn('callback_date', col('callback_date').cast(DateType()))\
.withColumn('ticket_close_date', col('ticket_close_date').cast(DateType()))\
.withColumn('ticket_tat', col('ticket_tat').cast('long'))
icrm_df=icrm_df.filter(col('interaction_date').isNotNull()).filter(col('source_system_customer_id').isNotNull())
icrm_df=icrm_df.na.fill(value=0,subset=["certificate_no"])
icrm_df=icrm_df.withColumn("certificate_no", coalesce(col("certificate_no"), lit('00000000')))

In [ ]:
icrm_df=icrm_df.select('source_app','interaction_type','interaction_date','policy_no',col('certificate_no'),'source_system_customer_id','reference_num','transaction_id','ticket_no',
'mobile','email','awb_no','customer_name','deposition','sub_deposition','callback_date','remarks','language_pref','language_barrier','lost_reason',
'competitor_name','ticket_close_date','ticket_tat','ticket_status','ticket_source','open_status','link_status','print_flag','courier','delivery_return_date',
'recieved_by')

In [ ]:
primary_key = ['source_app','interaction_type','source_system_customer_id', 'policy_no','certificate_no']

from pyspark.sql.window import Window

icrm_df = icrm_df.withColumn("rn", F.row_number()
        .over(Window.partitionBy(primary_key)
        .orderBy(F.col("interaction_date").desc())))

icrm_df = icrm_df.filter(F.col("rn") == 1).drop("rn")

In [ ]:
icrm_df.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
records=icrm_df.count()
records

In [ ]:
output_index = "interaction_journal_staging"
schema = "customermart"


import sys
try:

    icrm_df.write.format("greenplum")\
    .option("dbtable",output_index).option('dbschema',schema)\
    .option("server.port","1150-1170").option("url",prod_url)\
    .option("user", user_prod).option("password",pwd_prod).mode('overwrite').save()


except Exception as e:
    x = e
    print(x)
else:
    x = 200 #success
    print(x)

In [ ]:
table_name="interaction_journal"
conn_prod = psycopg2.connect(host=prod_host, port=prod_port, user=user_prod, password=pwd_prod, dbname=prod_dbname)
conn_to=conn_prod
status=''

import sys
try:
    update(conn_to,table_name,prod_gpdb_spark_options, schema="customermart")
except Exception as e:
    x = e
    print(x)
    status='Failed'
else:
    x = 'success'
    print(x)
    status='Success'

In [ ]:
table_name='interaction_journal'
source='ICRM'

if  x == 'success':
    update_progress(table_name,source,time_filter,records,start_time,starttime,'Success')
    print('SUCCESS')
else:
    update_progress(table_name,source,time_filter,'0',start_time,starttime,'Failed')
    print('FAILED')
    
icrm_df.unpersist()

# OCRM

In [ ]:
starttime = time.time()
start_time = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d %H:%M:%S.%f")
start_time

In [ ]:
table_name='interaction_journal'
source='OCRM'

In [ ]:
latest_progress=get_latest_progress(table_name,source)
# dates in string format
str_d1 = latest_progress.strftime("%Y-%m-%d")
str_d2 = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d")

# convert string to date object
d1 = datetime.datetime.strptime(str_d1, "%Y-%m-%d")
d2 = datetime.datetime.strptime(str_d2, "%Y-%m-%d")

# difference between dates in timedelta
delta = d2 - d1
day=delta.days-1#currently T-2 change 2 to 1 for T-1

time_filter={'start_date':latest_progress,'end_date':latest_progress+datetime.timedelta(days=day)}
time_filter

In [ ]:
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpspark",
"password": "spark@456",
"dbschema": "public",
"dbtable": "marketing_ocrm_mph_motor_renewal"}

ocrm_rpc = sqlContext.read.format("jdbc").options(**gscPythonOptions).load()\
.select('policy_no','right_party_contact_number','email_id','deleted','date_entered','date_modified')\
.filter((col('right_party_contact_number').isNotNull())\
        & (col('policy_no').isNotNull())\
        & (col('deleted')=='FALSE')).drop(col('deleted'))\
.filter(col('date_modified').between(to_timestamp(lit(time_filter['start_date']),
                                                                 format='yyyy-MM-dd'),
                                                    to_timestamp(lit(time_filter['end_date']),
                                                                 format='yyyy-MM-dd')))

In [ ]:
gscPythonOptions = {"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
                    "user": "gpspark",
                    "password": "spark@456",
                    "dbschema": "registers",
                    "dbtable": "premium_register",
                    "partitionColumn":"row_num",
                    "partitions": 16,
                    "server.port": '1150-1170'}

premium_reg = sqlContext.read.format("greenplum").options(**gscPythonOptions).load()\
                   .selectExpr("policy_no","certificate_no", 'num_reference_no as reference_num','customer_id', 'timestamp')\
                   .filter(col('customer_id').isNotNull())

premium_reg.columns

In [ ]:
ocrm_rpc=ocrm_rpc.join(premium_reg, ocrm_rpc.policy_no==premium_reg.policy_no, 'left').drop(premium_reg.policy_no)

In [ ]:
ocrm_rpc=ocrm_rpc.withColumn('source_app',lit('OCRM'))\
             .withColumn('interaction_type',lit('Outbound Call'))\
             .withColumn('interaction_date',col('date_entered'))\
             .withColumn('source_system_customer_id',col('customer_id'))\
             .withColumn('mobile',col('right_party_contact_number'))\
             .withColumn('email',col('email_id'))

In [ ]:
ocrm_rpc=ocrm_rpc.withColumn('transaction_id',lit(None).cast('string'))\
.withColumn('ticket_no',lit(None).cast('string'))\
.withColumn('awb_no',lit(None).cast('string'))\
.withColumn('customer_name',lit(None).cast('string'))\
.withColumn('deposition',lit(None).cast('string'))\
.withColumn('sub_deposition',lit(None).cast('string'))\
.withColumn('callback_date',lit(None).cast('date'))\
.withColumn('remarks',lit(None).cast('string'))\
.withColumn('language_pref',lit(None).cast('string'))\
.withColumn('language_barrier',lit(None).cast('string'))\
.withColumn('lost_reason',lit(None).cast('string'))\
.withColumn('competitor_name',lit(None).cast('string'))\
.withColumn('ticket_close_date',lit(None).cast('date'))\
.withColumn('ticket_tat',lit(None).cast('int'))\
.withColumn('ticket_status',lit(None).cast('string'))\
.withColumn('ticket_source',lit(None).cast('string'))\
.withColumn('open_status',lit(None).cast('string'))\
.withColumn('link_status',lit(None).cast('string'))\
.withColumn('print_flag',lit(None).cast('string'))\
.withColumn('courier',lit(None).cast('string'))\
.withColumn('delivery_return_date',lit(None).cast('date'))\
.withColumn('recieved_by',lit(None).cast('string'))

In [ ]:
ocrm_rpc=ocrm_rpc.select('source_app','interaction_type','interaction_date','policy_no','certificate_no','source_system_customer_id','reference_num','transaction_id','ticket_no',
'mobile','email','awb_no','customer_name','deposition','sub_deposition','callback_date','remarks','language_pref','language_barrier','lost_reason',
'competitor_name','ticket_close_date','ticket_tat','ticket_status','ticket_source','open_status','link_status','print_flag','courier','delivery_return_date',
'recieved_by')

In [ ]:
ocrm_rpc=ocrm_rpc.dropDuplicates()
ocrm_rpc=ocrm_rpc.withColumn("certificate_no", coalesce(col("certificate_no"), lit('00000000'))).filter(col('source_system_customer_id').isNotNull())

In [ ]:
primary_key = ['source_app','interaction_type','source_system_customer_id', 'policy_no','certificate_no']

from pyspark.sql.window import Window

ocrm_rpc = ocrm_rpc.withColumn("rn", F.row_number()
        .over(Window.partitionBy(primary_key)
        .orderBy(F.col("interaction_date").desc())))

ocrm_rpc = ocrm_rpc.filter(F.col("rn") == 1).drop("rn")

In [ ]:
ocrm_rpc.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
records=ocrm_rpc.count() ##223058
records

In [ ]:
output_index = "interaction_journal_staging" 
schema = "customermart"


import sys
try:

    ocrm_rpc.write.format("greenplum")\
    .option("dbtable",output_index).option('dbschema',schema)\
    .option("server.port","1150-1170").option("url",prod_url)\
    .option("user", user_prod).option("password",pwd_prod).mode('overwrite').save()


except Exception as e:
    x = e
    print(x)
else:
    x = 200 #success
    print(x)

In [ ]:
table_name="interaction_journal"
conn_prod = psycopg2.connect(host=prod_host, port=prod_port, user=user_prod, password=pwd_prod, dbname=prod_dbname)
conn_to=conn_prod
status=''

import sys
try:
    update(conn_to,table_name,prod_gpdb_spark_options, schema="customermart")
except Exception as e:
    x = e
    print(x)
    status='Failed'
else:
    x = 'success'
    print(x)
    status='Success'

In [ ]:
table_name='interaction_journal'
source='OCRM'

if  x == 'success':
    update_progress(table_name,source,time_filter,records,start_time,starttime,'Success')
    print('SUCCESS')
else:
    update_progress(table_name,source,time_filter,'0',start_time,starttime,'Failed')
    print('FAILED')
    
ocrm_rpc.unpersist()

# App ,broker,voice bot

In [8]:
starttime = time.time()
start_time = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d %H:%M:%S.%f")
start_time

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'2023-12-13 22:48:47.386415'

In [9]:
table_name='interaction_journal'
source='SALESFORCE_CASE'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
latest_progress=get_latest_progress(table_name,source)
# dates in string format
str_d1 = latest_progress.strftime("%Y-%m-%d")
str_d2 = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d")

# convert string to date object
d1 = datetime.datetime.strptime(str_d1, "%Y-%m-%d")
d2 = datetime.datetime.strptime(str_d2, "%Y-%m-%d")

# difference between dates in timedelta
delta = d2 - d1
day=delta.days-1#currently T-2 change 2 to 1 for T-1

time_filter={'start_date':latest_progress,'end_date':latest_progress+datetime.timedelta(days=day)}
time_filter

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'start_date': datetime.datetime(2023, 12, 7, 0, 0), 'end_date': datetime.datetime(2023, 12, 12, 0, 0)}

In [11]:
col_str=['requestor_primary_number__c as mobile','customer_code__c','reference_number_proposalnumber_quote_nu__c as reference_num1','spa_reference_number__c as reference_num2','sub_classification_disposition__c as sub_deposition',\
           'ticket_source__c as ticket_source','complaint_registration_date__c as interaction_date','closeddate as ticket_close_date','requestor_email_id__c as email',\
           'policy_number__c as policy_no','problem_remarks__c as remarks','status as open_status','sf_ticket_number__c as ticket_no','requester_name__c as customer_name',\
           'ticket_status__c as ticket_status','ageing_count_in_days__c as ticket_tat','certificate_number__c as certificate_no']
app_df=load_gpdb_jdbc(col_str,"salesforce_case",'public','load_date',time_filter).filter(col('policy_no').isNotNull())
app_df.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['mobile', 'customer_code__c', 'reference_num1', 'reference_num2', 'sub_deposition', 'ticket_source', 'interaction_date', 'ticket_close_date', 'email', 'policy_no', 'remarks', 'open_status', 'ticket_no', 'customer_name', 'ticket_status', 'ticket_tat', 'certificate_no']

In [12]:
app_df=app_df.withColumn('source_app',when(col('ticket_source').isin('Inbound Call'),lit('Salesforce Service CRM')).otherwise(col('ticket_source')))\
             .withColumn('interaction_type',col('ticket_source'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
app_df=app_df.withColumn("source_app", coalesce(col("source_app"), lit('Salesforce')))\
.withColumn("interaction_type", coalesce(col("interaction_type"), lit('Salesforce')))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
app_df=app_df.withColumn('reference_num',coalesce(col('reference_num1'),col('reference_num2')))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
app_df=app_df.withColumn('reference_num', col('reference_num').cast('long'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
gscPythonOptions = {
"url": "jdbc:postgresql://10.35.12.194:5432/gpadmin",
"user": "gpspark",
"password": "spark@456",
"dbtable": "datamarts.sales_all_details"}
sad = sqlContext.read.format("jdbc").options(**gscPythonOptions).load()\
       .selectExpr('proposal_no', 'ins_name_client_no as source_system_customer_id', 'policy_no', 'certificate_no')

sad.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['proposal_no', 'source_system_customer_id', 'policy_no', 'certificate_no']

In [17]:
sad_1=sad.select('proposal_no', 'source_system_customer_id', 'policy_no')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
app_df=app_df.join(sad_1, app_df.customer_code__c==sad_1.source_system_customer_id, 'inner').drop(sad_1.policy_no)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
app_df=app_df.withColumn('transaction_id',lit(None).cast('string'))\
.withColumn('awb_no',lit(None).cast('string'))\
.withColumn('deposition',lit(None).cast('string'))\
.withColumn('callback_date',lit(None).cast('date'))\
.withColumn('language_pref',lit(None).cast('string'))\
.withColumn('language_barrier',lit(None).cast('string'))\
.withColumn('lost_reason',lit(None).cast('string'))\
.withColumn('competitor_name',lit(None).cast('string'))\
.withColumn('link_status',lit(None).cast('string'))\
.withColumn('print_flag',lit(None).cast('string'))\
.withColumn('courier',lit(None).cast('string'))\
.withColumn('delivery_return_date',lit(None).cast('date'))\
.withColumn('recieved_by',lit(None).cast('string'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
from pyspark.sql.functions import unix_timestamp, from_unixtime
app_df=app_df.withColumn('interaction_date', from_unixtime(unix_timestamp('interaction_date', 'yyyy-MM-dd')).cast('timestamp'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
app_df=app_df.withColumn('interaction_date', col('interaction_date').cast(DateType()))\
.withColumn('delivery_return_date', col('delivery_return_date').cast(DateType()))\
.withColumn('reference_num', col('reference_num').cast('long'))\
.withColumn('callback_date', col('callback_date').cast(DateType()))\
.withColumn('ticket_close_date', col('ticket_close_date').cast(DateType()))\
.withColumn('ticket_tat', col('ticket_tat').cast('long'))
app_df=app_df.filter(col('interaction_date').isNotNull()).filter(col('source_system_customer_id').isNotNull())
app_df=app_df.withColumn("certificate_no", coalesce(col("certificate_no"), lit('00000000')))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
app_df=app_df.select('source_app','interaction_type','interaction_date','policy_no','certificate_no','source_system_customer_id','reference_num','transaction_id','ticket_no',
'mobile','email','awb_no','customer_name','deposition','sub_deposition','callback_date','remarks','language_pref','language_barrier','lost_reason',
'competitor_name','ticket_close_date','ticket_tat','ticket_status','ticket_source','open_status','link_status','print_flag','courier','delivery_return_date',
'recieved_by')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
app_df=app_df.dropDuplicates()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
primary_key = ['source_app','interaction_type','source_system_customer_id', 'policy_no','certificate_no']

from pyspark.sql.window import Window

app_df = app_df.withColumn("rn", F.row_number()
        .over(Window.partitionBy(primary_key)
        .orderBy(F.col("interaction_date").desc())))

app_df = app_df.filter(F.col("rn") == 1).drop("rn")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
app_df.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
records=app_df.count()
records

In [ ]:
output_index = "interaction_journal_staging"
schema = "customermart"


import sys
try:

    app_df.write.format("greenplum")\
    .option("dbtable",output_index).option('dbschema',schema)\
    .option("server.port","1150-1170").option("url",prod_url)\
    .option("user", user_prod).option("password",pwd_prod).mode('overwrite').save()


except Exception as e:
    x = e
    print(x)
else:
    x = 200 #success
    print(x)

In [ ]:
table_name="interaction_journal"
conn_prod = psycopg2.connect(host=prod_host, port=prod_port, user=user_prod, password=pwd_prod, dbname=prod_dbname)
conn_to=conn_prod
status=''

import sys
try:
    update(conn_to,table_name,prod_gpdb_spark_options, schema="customermart")
except Exception as e:
    x = e
    print(x)
    status='Failed'
else:
    x = 'success'
    print(x)
    status='Success'

In [ ]:
table_name='interaction_journal'
source='SALESFORCE_CASE'

if  x == 'success':
    update_progress(table_name,source,time_filter,records,start_time,starttime,'Success')
    print('SUCCESS')
else:
    update_progress(table_name,source,time_filter,'0',start_time,starttime,'Failed')
    print('FAILED')
    
app_df.unpersist()

# sales CRM

In [25]:
starttime = time.time()
start_time = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d %H:%M:%S.%f")
start_time

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'2023-12-13 23:04:04.906447'

In [26]:
table_name='interaction_journal'
source='CRM'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
latest_progress=get_latest_progress(table_name,source)
# dates in string format
str_d1 = latest_progress.strftime("%Y-%m-%d")
str_d2 = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d")

# convert string to date object
d1 = datetime.datetime.strptime(str_d1, "%Y-%m-%d")
d2 = datetime.datetime.strptime(str_d2, "%Y-%m-%d")

# difference between dates in timedelta
delta = d2 - d1
day=delta.days-1#currently T-2 change 2 to 1 for T-1

time_filter={'start_date':latest_progress,'end_date':latest_progress+datetime.timedelta(days=day)}
time_filter

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Executed_except ValueError('No records found!')
{'start_date': datetime.datetime(2001, 1, 1, 0, 0), 'end_date': datetime.datetime(2023, 12, 12, 0, 0)}

In [28]:
col_str=['policy_number__c','campaign_name__c','product__c','id',"lastmodifieddate" ]
gscPythonOptions = {
         "url": prod_url,
         "user": user_prod,
         "password": pwd_prod,
         "dbschema":"public",
         "dbtable": "salesforce_opportunity",
         "server.port":"1150-1170"} 
opportunity_df = sqlContext.read.format("jdbc").options(**gscPythonOptions).load()\
                .selectExpr(col_str)\
                .filter(col('policy_number__c').isNotNull())\
                .filter(col("load_date")>=(to_date(lit(time_filter['start_date']), format='yyyy-MM-dd')))\
                .filter(col("load_date")<=(to_date(lit(time_filter['end_date']), format='yyyy-MM-dd')))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
col_str=['Call_Back_Date_Time__c','Dispose_Date_Time__c','Parent_Status__c','Remarks__c as remarks','Sub_Status__c','Sub_Disposition__c as sub_deposition','whatid']
gscPythonOptions = {
         "url": prod_url,
         "user": user_prod,
         "password": pwd_prod,
         "dbschema":"public",
         "dbtable": "salesforce_activity",
         "server.port":"1150-1170"} 
activity_df = sqlContext.read.format("jdbc").options(**gscPythonOptions).load()\
                .selectExpr(col_str).filter(col('whatid').isNotNull())\
                .filter(col("load_date")>=(to_date(lit(time_filter['start_date']), format='yyyy-MM-dd')))\
                .filter(col("load_date")<=(to_date(lit(time_filter['end_date']), format='yyyy-MM-dd')))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
salescrm_df= opportunity_df.join(activity_df, opportunity_df.id == activity_df.whatid, "inner")\
.drop(activity_df.whatid)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
salescrm_df=salescrm_df.withColumn('source_app',lit('Salesforce Sales CRM')).withColumn('interaction_type',lit('Outbound Calling'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
salescrm_df=salescrm_df.withColumnRenamed('Dispose_Date_Time__c','interaction_date')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
salescrm_df.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['policy_number__c', 'campaign_name__c', 'product__c', 'id', 'lastmodifieddate', 'Call_Back_Date_Time__c', 'interaction_date', 'Parent_Status__c', 'remarks', 'Sub_Status__c', 'sub_deposition', 'source_app', 'interaction_type']

In [34]:
salescrm_df=salescrm_df.join(sad, salescrm_df.policy_number__c==sad.policy_no, 'inner').drop(sad.policy_no)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
salescrm_df=salescrm_df.withColumnRenamed('policy_number__c', 'policy_no')\
.withColumnRenamed('proposal_no', 'reference_num')\
.filter(col('interaction_date').isNotNull())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
salescrm_df.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['policy_no', 'campaign_name__c', 'product__c', 'id', 'lastmodifieddate', 'Call_Back_Date_Time__c', 'interaction_date', 'Parent_Status__c', 'remarks', 'Sub_Status__c', 'sub_deposition', 'source_app', 'interaction_type', 'reference_num', 'source_system_customer_id', 'certificate_no']

In [37]:
salescrm_df=salescrm_df.withColumn('transaction_id',lit(None).cast('string'))\
.withColumn('ticket_no',lit(None).cast('string'))\
.withColumn('mobile',lit(None).cast('string'))\
.withColumn('email',lit(None).cast('string'))\
.withColumn('awb_no',lit(None).cast('string'))\
.withColumn('customer_name',lit(None).cast('string'))\
.withColumn('deposition',lit(None).cast('string'))\
.withColumn('callback_date',lit(None).cast('date'))\
.withColumn('language_pref',lit(None).cast('string'))\
.withColumn('language_barrier',lit(None).cast('string'))\
.withColumn('lost_reason',lit(None).cast('string'))\
.withColumn('competitor_name',lit(None).cast('string'))\
.withColumn('ticket_close_date',lit(None).cast('date'))\
.withColumn('ticket_tat',lit(None).cast('int'))\
.withColumn('ticket_status',lit(None).cast('string'))\
.withColumn('ticket_source',lit(None).cast('string'))\
.withColumn('open_status',lit(None).cast('string'))\
.withColumn('link_status',lit(None).cast('string'))\
.withColumn('print_flag',lit(None).cast('string'))\
.withColumn('courier',lit(None).cast('string'))\
.withColumn('delivery_return_date',lit(None).cast('date'))\
.withColumn('recieved_by',lit(None).cast('string'))
#salescrm_df=salescrm_df.select([col(c).cast("string") for c in salescrm_df.columns])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
salescrm_df=salescrm_df.select('source_app','interaction_type','interaction_date','policy_no','certificate_no','source_system_customer_id','reference_num','transaction_id','ticket_no',
'mobile','email','awb_no','customer_name','deposition','sub_deposition','callback_date','remarks','language_pref','language_barrier','lost_reason',
'competitor_name','ticket_close_date','ticket_tat','ticket_status','ticket_source','open_status','link_status','print_flag','courier','delivery_return_date',
'recieved_by')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:

salescrm_df=salescrm_df.withColumn('interaction_date', col('interaction_date').cast(DateType()))\
.withColumn('delivery_return_date', col('delivery_return_date').cast(DateType()))\
.withColumn('reference_num', col('reference_num').cast('long'))\
.withColumn('callback_date', col('callback_date').cast(DateType()))\
.withColumn('ticket_close_date', col('ticket_close_date').cast(DateType()))\
.withColumn('ticket_tat', col('ticket_tat').cast('long'))
salescrm_df=salescrm_df.filter(col('interaction_date').isNotNull()).filter(col('source_system_customer_id').isNotNull())
salescrm_df=salescrm_df.withColumn("certificate_no", coalesce(col("certificate_no"), lit('00000000')))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
primary_key = ['source_app','interaction_type','source_system_customer_id', 'policy_no','certificate_no']

from pyspark.sql.window import Window

salescrm_df = salescrm_df.withColumn("rn", F.row_number()
        .over(Window.partitionBy(primary_key)
        .orderBy(F.col("interaction_date").desc())))

salescrm_df = salescrm_df.filter(F.col("rn") == 1).drop("rn")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
salescrm_df.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
records=salescrm_df.count()
records

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '400' from http://10.33.195.18:8998/sessions/15982/statements/63 with error payload: {"msg":"requirement failed: Session isn't active."}


In [64]:
output_index = "interaction_journal_staging" 
schema = "customermart"


import sys
try:

    salescrm_df.write.format("greenplum")\
    .option("dbtable",output_index).option('dbschema',schema)\
    .option("server.port","1150-1170").option("url",prod_url)\
    .option("user", user_prod).option("password",pwd_prod).mode('overwrite').save()


except Exception as e:
    x = e
    print(x)
else:
    x = 200 #success
    print(x)

An error was encountered:
Session 15982 did not reach idle status in time. Current status is shutting_down.


In [ ]:
table_name="interaction_journal"
conn_prod = psycopg2.connect(host=prod_host, port=prod_port, user=user_prod, password=pwd_prod, dbname=prod_dbname)
conn_to=conn_prod
status=''

import sys
try:
    update(conn_to,table_name,prod_gpdb_spark_options, schema="customermart")
except Exception as e:
    x = e
    print(x)
    status='Failed'
else:
    x = 'success'
    print(x)
    status='Success'

In [ ]:
table_name='interaction_journal'
source='SALESCRM'

if  x == 'success':
    update_progress(table_name,source,time_filter,records,start_time,starttime,'Success')
    print('SUCCESS')
else:
    update_progress(table_name,source,time_filter,'0',start_time,starttime,'Failed')
    print('FAILED')
    
salescrm_df.unpersist()

# ACM 

In [41]:
starttime = time.time()
start_time = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d %H:%M:%S.%f")
start_time

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'2023-12-13 23:04:28.870252'

In [42]:
table_name='interaction_journal'
source='ACM'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
latest_progress=get_latest_progress(table_name,source)
# dates in string format
str_d1 = latest_progress.strftime("%Y-%m-%d")
str_d2 = datetime.datetime.now(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d")

# convert string to date object
d1 = datetime.datetime.strptime(str_d1, "%Y-%m-%d")
d2 = datetime.datetime.strptime(str_d2, "%Y-%m-%d")

# difference between dates in timedelta
delta = d2 - d1
day=delta.days-1#currently T-2 change 2 to 1 for T-1

time_filter={'start_date':latest_progress,'end_date':latest_progress+datetime.timedelta(days=day)}
time_filter

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'start_date': datetime.datetime(2023, 9, 25, 0, 0), 'end_date': datetime.datetime(2023, 12, 12, 0, 0)}

In [44]:
gscPythonOptions = {
         "url": prod_url,
         "user": user_prod,
         "password": pwd_prod,
         "dbschema":"public",
         "dbtable": "adobe_bike_renewal_report",
         "server.port":"1150-1170"} 

# Get the data  from Table

acm_bike_df = sqlContext.read.format("jdbc").options(**gscPythonOptions).load()\
.selectExpr('policy_number as policy_no','mode as interaction_type','deliver_time as interaction_date','delivery_status','click_open_status as open_status').dropDuplicates()\
.filter(col("load_date")>=(to_date(lit(time_filter['start_date']), format='yyyy-MM-dd')))\
.filter(col("load_date")<=(to_date(lit(time_filter['end_date']), format='yyyy-MM-dd')))\
.filter(col('interaction_date').isNotNull())

acm_bike_df=acm_bike_df.withColumn('source_app',lit('ACM Mobile/Email'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
acm_bike_df=acm_bike_df.withColumn('transaction_id',lit(None).cast('string'))\
.withColumn('ticket_no',lit(None).cast('string'))\
.withColumn('mobile',lit(None).cast('string'))\
.withColumn('email',lit(None).cast('string'))\
.withColumn('awb_no',lit(None).cast('string'))\
.withColumn('customer_name',lit(None).cast('string'))\
.withColumn('deposition',lit(None).cast('string'))\
.withColumn('sub_deposition',lit(None).cast('string'))\
.withColumn('callback_date',lit(None).cast('date'))\
.withColumn('remarks',lit(None).cast('string'))\
.withColumn('language_pref',lit(None).cast('string'))\
.withColumn('language_barrier',lit(None).cast('string'))\
.withColumn('lost_reason',lit(None).cast('string'))\
.withColumn('competitor_name',lit(None).cast('string'))\
.withColumn('ticket_close_date',lit(None).cast('date'))\
.withColumn('ticket_tat',lit(None).cast('int'))\
.withColumn('ticket_status',lit(None).cast('string'))\
.withColumn('ticket_source',lit(None).cast('string'))\
.withColumn('link_status',lit(None).cast('string'))\
.withColumn('print_flag',lit(None).cast('string'))\
.withColumn('courier',lit(None).cast('string'))\
.withColumn('delivery_return_date',lit(None).cast('date'))\
.withColumn('recieved_by',lit(None).cast('string'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
acm_bike_df=acm_bike_df.select('source_app','interaction_type','interaction_date','policy_no','transaction_id','ticket_no',
'mobile','email','awb_no','customer_name','deposition','sub_deposition','callback_date','remarks','language_pref','language_barrier','lost_reason',
'competitor_name','ticket_close_date','ticket_tat','ticket_status','ticket_source','open_status','link_status','print_flag','courier','delivery_return_date',
'recieved_by')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
gscPythonOptions = {
         "url": prod_url,
         "user": user_prod,
         "password": pwd_prod,
         "dbschema":"public",
         "dbtable": "adobe_cv_renewal_report",
         "server.port":"1150-1170"} 

# Get the data  from credit_bureau_data Table

acm_cv_df = sqlContext.read.format("jdbc").options(**gscPythonOptions).load()\
.selectExpr('policy_number as policy_no','mode as interaction_type','deliver_time as interaction_date','delivery_status','click_open_status as open_status').dropDuplicates()\
.filter(col("load_date")>=(to_date(lit(time_filter['start_date']), format='yyyy-MM-dd')))\
.filter(col("load_date")<=(to_date(lit(time_filter['end_date']), format='yyyy-MM-dd')))\
.filter(col('interaction_date').isNotNull())
#acm_cv_df.columns

acm_cv_df=acm_cv_df.withColumn('source_app',lit('ACM Mobile/Email'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
acm_cv_df=acm_cv_df.withColumn('transaction_id',lit(None).cast('string'))\
.withColumn('ticket_no',lit(None).cast('string'))\
.withColumn('mobile',lit(None).cast('string'))\
.withColumn('email',lit(None).cast('string'))\
.withColumn('awb_no',lit(None).cast('string'))\
.withColumn('customer_name',lit(None).cast('string'))\
.withColumn('deposition',lit(None).cast('string'))\
.withColumn('sub_deposition',lit(None).cast('string'))\
.withColumn('callback_date',lit(None).cast('date'))\
.withColumn('remarks',lit(None).cast('string'))\
.withColumn('language_pref',lit(None).cast('string'))\
.withColumn('language_barrier',lit(None).cast('string'))\
.withColumn('lost_reason',lit(None).cast('string'))\
.withColumn('competitor_name',lit(None).cast('string'))\
.withColumn('ticket_close_date',lit(None).cast('date'))\
.withColumn('ticket_tat',lit(None).cast('int'))\
.withColumn('ticket_status',lit(None).cast('string'))\
.withColumn('ticket_source',lit(None).cast('string'))\
.withColumn('link_status',lit(None).cast('string'))\
.withColumn('print_flag',lit(None).cast('string'))\
.withColumn('courier',lit(None).cast('string'))\
.withColumn('delivery_return_date',lit(None).cast('date'))\
.withColumn('recieved_by',lit(None).cast('string'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
acm_cv_df=acm_cv_df.select('source_app','interaction_type','interaction_date','policy_no','transaction_id','ticket_no',
'mobile','email','awb_no','customer_name','deposition','sub_deposition','callback_date','remarks','language_pref','language_barrier','lost_reason',
'competitor_name','ticket_close_date','ticket_tat','ticket_status','ticket_source','open_status','link_status','print_flag','courier','delivery_return_date',
'recieved_by')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
gscPythonOptions = {
         "url": prod_url,
         "user": user_prod,
         "password": pwd_prod,
         "dbschema":"public",
         "dbtable": "adobe_health_renewal_report",
         "server.port":"1150-1170"} 

# Get the data  from credit_bureau_data Table

acm_health_df = sqlContext.read.format("jdbc").options(**gscPythonOptions).load()\
.selectExpr('policy_number as policy_no','mode as interaction_type','deliver_time as interaction_date','delivery_status','click_open_status as open_status').dropDuplicates()\
.filter(col("load_date")>=(to_date(lit(time_filter['start_date']), format='yyyy-MM-dd')))\
.filter(col("load_date")<=(to_date(lit(time_filter['end_date']), format='yyyy-MM-dd')))\
.filter(col('interaction_date').isNotNull())\
.filter(col('policy_no').isNotNull())
#acm_health_df.columns

acm_health_df=acm_health_df.withColumn('source_app',lit('ACM Mobile/Email'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
acm_health_df=acm_health_df.withColumn('transaction_id',lit(None).cast('string'))\
.withColumn('ticket_no',lit(None).cast('string'))\
.withColumn('mobile',lit(None).cast('string'))\
.withColumn('email',lit(None).cast('string'))\
.withColumn('awb_no',lit(None).cast('string'))\
.withColumn('customer_name',lit(None).cast('string'))\
.withColumn('deposition',lit(None).cast('string'))\
.withColumn('sub_deposition',lit(None).cast('string'))\
.withColumn('callback_date',lit(None).cast('date'))\
.withColumn('remarks',lit(None).cast('string'))\
.withColumn('language_pref',lit(None).cast('string'))\
.withColumn('language_barrier',lit(None).cast('string'))\
.withColumn('lost_reason',lit(None).cast('string'))\
.withColumn('competitor_name',lit(None).cast('string'))\
.withColumn('ticket_close_date',lit(None).cast('date'))\
.withColumn('ticket_tat',lit(None).cast('int'))\
.withColumn('ticket_status',lit(None).cast('string'))\
.withColumn('ticket_source',lit(None).cast('string'))\
.withColumn('link_status',lit(None).cast('string'))\
.withColumn('print_flag',lit(None).cast('string'))\
.withColumn('courier',lit(None).cast('string'))\
.withColumn('delivery_return_date',lit(None).cast('date'))\
.withColumn('recieved_by',lit(None).cast('string'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
acm_health_df=acm_health_df.select('source_app','interaction_type','interaction_date','policy_no','transaction_id','ticket_no',
'mobile','email','awb_no','customer_name','deposition','sub_deposition','callback_date','remarks','language_pref','language_barrier','lost_reason',
'competitor_name','ticket_close_date','ticket_tat','ticket_status','ticket_source','open_status','link_status','print_flag','courier','delivery_return_date',
'recieved_by')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
df_union_acm_1=acm_bike_df.union(acm_cv_df)
df_union_acm_2=df_union_acm_1.union(acm_health_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
df_union_acm_2=df_union_acm_2.join(sad, df_union_acm_2.policy_no==sad.policy_no, 'inner').drop(sad.policy_no)\
.withColumnRenamed('proposal_no', 'reference_num')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
from pyspark.sql.functions import unix_timestamp, from_unixtime
df_union_acm_2=df_union_acm_2.withColumn('interaction_date', from_unixtime(unix_timestamp('interaction_date', 'yyyy/MM/dd HH:mm:ss')).cast('timestamp'))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
df_union_acm_2=df_union_acm_2.withColumn('interaction_date', col('interaction_date').cast(DateType()))\
.withColumn('delivery_return_date', col('delivery_return_date').cast(DateType()))\
.withColumn('reference_num', col('reference_num').cast('long'))\
.withColumn('callback_date', col('callback_date').cast(DateType()))\
.withColumn('ticket_close_date', col('ticket_close_date').cast(DateType()))\
.withColumn('ticket_tat', col('ticket_tat').cast('long'))
df_union_acm_2=df_union_acm_2.filter(col('source_system_customer_id').isNotNull()).filter(col('interaction_date').isNotNull())
df_union_acm_2=df_union_acm_2.withColumn("certificate_no", coalesce(col("certificate_no"), lit('00000000')))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
df_union_acm_2=df_union_acm_2.select('source_app','interaction_type','interaction_date','policy_no','certificate_no','source_system_customer_id','reference_num','transaction_id','ticket_no',
'mobile','email','awb_no','customer_name','deposition','sub_deposition','callback_date','remarks','language_pref','language_barrier','lost_reason',
'competitor_name','ticket_close_date','ticket_tat','ticket_status','ticket_source','open_status','link_status','print_flag','courier','delivery_return_date',
'recieved_by')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
primary_key = ['source_app','interaction_type','source_system_customer_id', 'policy_no','certificate_no']

from pyspark.sql.window import Window

df_union_acm_2 = df_union_acm_2.withColumn("rn", F.row_number()
        .over(Window.partitionBy(primary_key)
        .orderBy(F.col("interaction_date").desc())))

df_union_acm_2 = df_union_acm_2.filter(F.col("rn") == 1).drop("rn")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
df_union_acm_2.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
records=df_union_acm_2.count()
records

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [60]:
output_index = "interaction_journal_staging" 
schema = "customermart"


import sys
try:

    df_union_acm_2.write.format("greenplum")\
    .option("dbtable",output_index).option('dbschema',schema)\
    .option("server.port","1150-1170").option("url",prod_url)\
    .option("user", user_prod).option("password",pwd_prod).mode('overwrite').save()


except Exception as e:
    x = e
    print(x)
else:
    x = 200 #success
    print(x)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

200

In [61]:
table_name="interaction_journal"
conn_prod = psycopg2.connect(host=prod_host, port=prod_port, user=user_prod, password=pwd_prod, dbname=prod_dbname)
conn_to=conn_prod
status=''

import sys
try:
    update(conn_to,table_name,prod_gpdb_spark_options, schema="customermart")
except Exception as e:
    x = e
    print(x)
    status='Failed'
else:
    x = 'success'
    print(x)
    status='Success'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Inside try segment
Executed Only insert query
success

In [62]:
table_name='interaction_journal'
source='ACM'

if  x == 'success':
    update_progress(table_name,source,time_filter,records,start_time,starttime,'Success')
    print('SUCCESS')
else:
    update_progress(table_name,source,time_filter,'0',start_time,starttime,'Failed')
    print('FAILED')
    
df_union_acm_2.unpersist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Updated Progress for interaction_journal from ACM until 2023-12-12 00:00:00. 0 records processed in this run
success
SUCCESS
DataFrame[source_app: string, interaction_type: string, interaction_date: date, policy_no: string, certificate_no: string, source_system_customer_id: string, reference_num: bigint, transaction_id: string, ticket_no: string, mobile: string, email: string, awb_no: string, customer_name: string, deposition: string, sub_deposition: string, callback_date: date, remarks: string, language_pref: string, language_barrier: string, lost_reason: string, competitor_name: string, ticket_close_date: date, ticket_tat: bigint, ticket_status: string, ticket_source: string, open_status: string, link_status: string, print_flag: string, courier: string, delivery_return_date: date, recieved_by: string]

In [ ]:
spark.stop()